In [1]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import locale
from locale import atof

In [2]:
# Load in csv
filepath = os.path.join("CSV Files\Zillow CSV\Price.csv")
price_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\Zillow CSV\Rent.csv")
rent_df = pd.read_csv(filepath)

In [28]:
# Convert Region Name into Seperate City and State Columns
rent_df['City'],rent_df['State'] = rent_df["RegionName"].str.split(',', 2).str[0:2].str
rent_df['State'].replace(' TX','TX', inplace=True)
price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
price_df['State'].replace(' TX','TX', inplace=True)

# Extract only the Texas Data
rent_tx_df = rent_df.loc[rent_df["State"] == "TX", :]
rent_tx_df = rent_tx_df.set_index('City')
price_tx_df = price_df.loc[price_df["State"] == "TX", :]
price_tx_df = price_tx_df.set_index('City')

# Remove Undesired Columns
rent_tx_df = rent_tx_df.drop(rent_tx_df.columns[[0,1,2,-1]], axis=1)
price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)

# Transpose Data Frame
rent = rent_tx_df.transpose()
price = price_tx_df.transpose()

#Move the Index
rent.reset_index(level=0, inplace=True)
rent = rent.rename(columns = {'index':'Date'})
rent = rent.reset_index(drop=True)
price.reset_index(level=0, inplace=True)
price = price.rename(columns = {'index':'Date'})
price = price.reset_index(drop=True)

# Convert the Date into a Month and Year Column
rent['Year'],rent['Month'] = rent["Date"].str.split('-', 2).str[0:2].str
rent = rent.drop(rent.columns[[0]], axis=1)
price['Year'],price['Month'] = price["Date"].str.split('-', 2).str[0:2].str
price = price.drop(price.columns[[0]], axis=1)

# Melt Data Frames and Restructure and Merge
rent_melt = pd.melt(rent, id_vars=["Year", "Month"])
rent_melt = rent_melt.rename(columns={'value': 'Rent'})
rent_melt = rent_melt.replace('Dallas-Fort Worth', 'Dallas')
price_melt = pd.melt(price, id_vars=["Year", "Month"])
price_melt = price_melt.rename(columns={'value': 'House Price'})
merge_melts = pd.merge(rent_melt,price_melt, how='outer', on=['Year','Month','City'])
merge_melts.head()

,Year,Month,City,Rent,House Price
0,2010,11,Dallas,1295.0,165000.0
1,2010,12,Dallas,1299.0,159900.0
2,2011,01,Dallas,1303.0,159000.0
3,2011,02,Dallas,1307.0,159000.0
4,2011,03,Dallas,1308.0,162990.0
